# This notebook shows how to Hydra layer2 connection works 

## Configure the Fabric Environment

In [7]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_PROJECT_ID']='b3cffedd-ddb4-43ee-b57d-459b768e14ca'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='xai_0051546236'
#os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/xai-bastion-1'
# Set the keypair FABRIC will install in your slice.
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/.ssh/id_rsa.pub'

# If your private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Import FABRIC API

In [8]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

In [3]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
TACC        10  178/320  1992/2560  112144/116400  627/635                 1/2                             4/4                            16/16               4/4               6/6
UTAH        10  249/320  2276/2560  114267/116400  621/635                 2/2                             4/4                            16/16               4/4               5/5
WASH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2

## Create Slice


### Configure Slice Parameters



In [9]:
slice_name = 'Hydra_L2-diff-sits'
site1 = 'UTAH'
site2 = 'STAR'
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
node4_name = 'Node4'
node5_name = 'Node5'
network_name=['net1','net2','net3','net4','net5','net6','net7','net8','net9','net10']
node1_nic_name = ['nic1_1', 'nic1_2','nic1_3', 'nic1_4']
node2_nic_name = ['nic2_1','nic2_2', 'nic2_3', 'nic2_4' ]
node3_nic_name = ['nic3_1','nic3_2', 'nic3_3', 'nic3_4' ]
node4_nic_name = ['nic4_1','nic4_2', 'nic4_3', 'nic4_4' ]
node5_nic_name = ['nic5_1','nic5_2', 'nic5_3', 'nic5_4' ]
image = 'default_ubuntu_20'
cores = 2
ram = 8
disk = 50

In [21]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1_1 = node1.add_component(model='NIC_Basic', name=node1_nic_name[0]).get_interfaces()[0]
    iface1_2 = node1.add_component(model='NIC_Basic', name=node1_nic_name[1]).get_interfaces()[0]
    iface1_3 = node1.add_component(model='NIC_Basic', name=node1_nic_name[2]).get_interfaces()[0]
    iface1_4 = node1.add_component(model='NIC_Basic', name=node1_nic_name[3]).get_interfaces()[0]
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2_1 = node2.add_component(model='NIC_Basic', name=node2_nic_name[0]).get_interfaces()[0]
    iface2_2 = node2.add_component(model='NIC_Basic', name=node2_nic_name[1]).get_interfaces()[0]
    iface2_3 = node2.add_component(model='NIC_Basic', name=node2_nic_name[2]).get_interfaces()[0]
    iface2_4 = node2.add_component(model='NIC_Basic', name=node2_nic_name[3]).get_interfaces()[0]
    
    # Node3
    node3 = slice.add_node(name=node3_name, site=site2)
    node3.set_capacities(cores=cores, ram=ram, disk=disk)
    node3.set_image(image)
    iface3_1 = node3.add_component(model='NIC_Basic', name=node3_nic_name[0]).get_interfaces()[0]
    iface3_2 = node3.add_component(model='NIC_Basic', name=node3_nic_name[1]).get_interfaces()[0]
    iface3_3 = node3.add_component(model='NIC_Basic', name=node3_nic_name[2]).get_interfaces()[0]
    iface3_4 = node3.add_component(model='NIC_Basic', name=node3_nic_name[3]).get_interfaces()[0]
    
    # Node4
    node4 = slice.add_node(name=node4_name, site=site2)
    node4.set_capacities(cores=cores, ram=ram, disk=disk)
    node4.set_image(image)
    iface4_1 = node4.add_component(model='NIC_Basic', name=node4_nic_name[0]).get_interfaces()[0]
    iface4_2 = node4.add_component(model='NIC_Basic', name=node4_nic_name[1]).get_interfaces()[0]
    iface4_3 = node4.add_component(model='NIC_Basic', name=node4_nic_name[2]).get_interfaces()[0]
    iface4_4 = node4.add_component(model='NIC_Basic', name=node4_nic_name[3]).get_interfaces()[0]
    
    # Node5
    node5 = slice.add_node(name=node5_name, site=site2)
    node5.set_capacities(cores=cores, ram=ram, disk=disk)
    node5.set_image(image)
    iface5_1 = node5.add_component(model='NIC_Basic', name=node5_nic_name[0]).get_interfaces()[0]
    iface5_2 = node5.add_component(model='NIC_Basic', name=node5_nic_name[1]).get_interfaces()[0]
    iface5_3 = node5.add_component(model='NIC_Basic', name=node5_nic_name[2]).get_interfaces()[0]
    iface5_4 = node5.add_component(model='NIC_Basic', name=node5_nic_name[3]).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name[0], interfaces=[iface1_1, iface2_1])
    net2 = slice.add_l2network(name=network_name[1], interfaces=[iface1_2, iface3_1])
    net3 = slice.add_l2network(name=network_name[2], interfaces=[iface1_3, iface4_1])
    net4 = slice.add_l2network(name=network_name[3], interfaces=[iface1_4, iface5_1])
    net5 = slice.add_l2network(name=network_name[4], interfaces=[iface2_2, iface3_2])
    net6 = slice.add_l2network(name=network_name[5], interfaces=[iface2_3, iface4_2])
    net7 = slice.add_l2network(name=network_name[6], interfaces=[iface2_4, iface5_2])
    net8 = slice.add_l2network(name=network_name[7], interfaces=[iface3_3, iface4_3])
    net9 = slice.add_l2network(name=network_name[8], interfaces=[iface3_4, iface5_3])
    net10 = slice.add_l2network(name=network_name[9], interfaces=[iface4_4, iface5_4])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Fail: {e}")


-----------  ------------------------------------
Slice Name   Hydra_L2-diff-sits
Slice ID     53ad87db-2e9c-4beb-b794-646662c01228
Slice State  StableOK
Lease End    2022-06-08 20:20:54 +0000
-----------  ------------------------------------

Retry: 22, Time: 301 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
44a14e4a-b90f-4057-bcbc-1cbfede5b054  Node1   UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fefd:76d5     Active
258eb24e-36ec-4ea5-ae48-9af0081e25d6  Node2   STAR    star-w4.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:400:a100:3030:f816:3eff:fe7f:53c0  Active
885503e5-6f1f-4472-b032

## Get the Slice

In [22]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   Hydra_L2-diff-sits
Slice ID     53ad87db-2e9c-4beb-b794-646662c01228
Slice State  StableOK
Lease End    2022-06-08 20:20:54 +0000
-----------  ------------------------------------


In [23]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
44a14e4a-b90f-4057-bcbc-1cbfede5b054  Node1   UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fefd:76d5     Active
258eb24e-36ec-4ea5-ae48-9af0081e25d6  Node2   STAR    star-w4.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:400:a100:3030:f816:3eff:fe7f:53c0  Active
885503e5-6f1f-4472-b032-50d133f4c3e6  Node3   STAR    star-w4.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:400:a100:3030:f816:3eff:fef7:e831  Active
a44ff200-9952-4937-ba49-3e3502ca425e  Node4   STAR    star-w4.fabric-testbed.net        2      8     100  default_ubuntu_2

In [25]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------
ID                 44a14e4a-b90f-4057-bcbc-1cbfede5b054
Name               Node1
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               utah-w5.fabric-testbed.net
Site               UTAH
Management IP      2001:1948:417:7:f816:3eff:fefd:76d5
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/.ssh/id_rsa -J xai_0051546236@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fefd:76d5
-----------------  ------------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------------------
ID                 258eb24e-36ec-4ea5-ae

### interface 

In [26]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name             Node    Network      Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
---------------  ------  ---------  -----------  ------  -----------------  -----------------------  --------------
Node1-nic1_3-p1  Node1   net3                 0          06:E6:02:2D:02:57  ens8                     ens8
Node1-nic1_1-p1  Node1   net1                 0          06:1F:9F:D4:E2:30  ens7                     ens7
Node1-nic1_4-p1  Node1   net4                 0          0A:5C:7E:54:66:E7  ens9                     ens9
Node1-nic1_2-p1  Node1   net2                 0          0A:CE:D5:B5:F0:09  ens10                    ens10
Node2-nic2_1-p1  Node2   net1                 0          26:84:F9:A7:01:D7  ens7                     ens7
Node2-nic2_4-p1  Node2   net7                 0          26:9B:CA:79:F5:4C  ens8                     ens8
Node2-nic2_2-p1  Node2   net5                 0          26:CB:8B:88:5B:C0  ens9                     ens9
Node2-nic2_3-p1  Node2   ne

In [29]:
## environment set up 
# necessary packages
try:
    for node in slice.get_nodes():
        stdout, stderr = node.execute('echo "deb [trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main" | sudo tee /etc/apt/sources.list.d/nfd-nightly.list')
        print(stdout)
        stdout, stderr = node.execute('sudo apt update && sudo apt upgrade -y')              
        #print(stdout)
        stdout, stderr = node.execute('sudo apt -y install python3-pip libndn-cxx-dev nfd ndnping ndnpeek ndn-dissect ndnchunks ndnsec')
        #print(stdout)
        stdout, stderr = node.execute('sudo pip3 install python-ndn ndn-storage ndn-svs ndn-hydra')
        #print(stdout)
        stdout, stderr = node.execute('sudo apt install net-tools')
        #
        stdout, stderr = node.execute('sudo apt update && sudo apt upgrade -y')
        #print(stdout)
except Exception as e:
    print(f"Exception: {e}")

deb [trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main

deb [trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main

deb [trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main

deb [trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main

deb [trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main



In [30]:
# face and route node1
try:
    stdout, stderr = slice.get_nodes()[0].execute('nfd-start')
    print(stdout)
    stdout, stderr = slice.get_nodes()[0].execute('nfdc face create remote ether://[26:84:F9:A7:01:D7] local dev://ens7' )              
    print(stdout)
    stdout, stderr = slice.get_nodes()[0].execute('nfdc route add / ether://[26:84:F9:A7:01:D7]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[0].execute('ndnsec key-gen /hydra/node1 | ndnsec cert-install -')
    print(stdout)                               
except Exception as e:
    print(f"Exception: {e}")


face-created id=270 local=dev://ens7 remote=ether://[26:84:f9:a7:01:d7] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/ nexthop=270 origin=static cost=0 flags=child-inherit expires=never




In [31]:
# face and route node2
try:
    stdout, stderr = slice.get_nodes()[1].execute('nfd-start')
    print(stdout)
    print("Node 3")
    stdout, stderr = slice.get_nodes()[1].execute('nfdc face create remote ether://[36:A0:EB:D1:F9:D8] local dev://ens9')              
    print(stdout)
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /hydra/group ether://[36:A0:EB:D1:F9:D8]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /hydra/node/node3 ether://[36:A0:EB:D1:F9:D8]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /node3 ether://[36:A0:EB:D1:F9:D8]')              
    print(stdout)
    
    print("Node 4")
    stdout, stderr = slice.get_nodes()[1].execute('nfdc face create remote ether://[4E:02:43:B0:AD:C2] local dev://ens10 ')
    print(stdout)
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /hydra/group ether://[4E:02:43:B0:AD:C2]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /hydra/node/node4 ether://[4E:02:43:B0:AD:C2]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /node4 ether://[4E:02:43:B0:AD:C2]')
    print(stdout)
    
    print("Node 5")
    stdout, stderr = slice.get_nodes()[1].execute('nfdc face create remote ether://[5A:EB:92:4F:37:D9] local dev://ens8 ')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /hydra/group ether://[5A:EB:92:4F:37:D9]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /hydra/node/node5 ether://[5A:EB:92:4F:37:D9]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /node5 ether://[5A:EB:92:4F:37:D9]')
    print(stdout)
    
    print("Node 1")
    stdout, stderr = slice.get_nodes()[1].execute('nfdc strategy set /hydra/group /localhost/nfd/strategy/multicast')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[1].execute('nfdc face create ether://[06:1F:9F:D4:E2:30] local dev://ens7 ')
    print(stdout)
    stdout, stderr = slice.get_nodes()[1].execute('nfdc route add /client ether://[06:1F:9F:D4:E2:30]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[1].execute('ndnsec key-gen /hydra/node2 | ndnsec cert-install -')
    print(stdout) 
except Exception as e:
    print(f"Exception: {e}")


Node 3
face-created id=270 local=dev://ens9 remote=ether://[36:a0:eb:d1:f9:d8] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node3 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/node3 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

Node 4
face-created id=275 local=dev://ens10 remote=ether://[4e:02:43:b0:ad:c2] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=275 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node4 nexthop=275 origin=static cost=0 flags=child-inherit expires=never

route-add-accepte

In [37]:
# face and route node3
try:
    stdout, stderr = slice.get_nodes()[2].execute('nfd-start')
    print(stdout)
    print("Node 2")
    stdout, stderr = slice.get_nodes()[2].execute('nfdc face create remote ether://[26:CB:8B:88:5B:C0] local dev://ens7')              
    print(stdout)
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /hydra/group ether://[26:CB:8B:88:5B:C0]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /hydra/node/node2 ether://[26:CB:8B:88:5B:C0]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /node2 ether://[26:CB:8B:88:5B:C0]')              
    print(stdout)
    
    print("Node 4")
    stdout, stderr = slice.get_nodes()[2].execute('nfdc face create remote ether://[4E:CA:E2:44:72:54] local dev://ens9 ')
    print(stdout)
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /hydra/group ether://[4E:CA:E2:44:72:54]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /hydra/node/node4 ether://[4E:CA:E2:44:72:54]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /node4 ether://[4E:CA:E2:44:72:54]')
    print(stdout)
    
    print("Node 5")
    stdout, stderr = slice.get_nodes()[2].execute('nfdc face create remote ether://[5E:3D:CF:D4:D9:74] local dev://ens10')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /hydra/group ether://[5E:3D:CF:D4:D9:74]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /hydra/node/node5 ether://[5E:3D:CF:D4:D9:74]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /node5 ether://[5E:3D:CF:D4:D9:74]')
    print(stdout)
    
    print("Node 1")
    stdout, stderr = slice.get_nodes()[2].execute('nfdc strategy set /hydra/group /localhost/nfd/strategy/multicast')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[2].execute('nfdc face create ether://[0A:5C:7E:54:66:E7] local dev://ens8 ')
    print(stdout)
    stdout, stderr = slice.get_nodes()[2].execute('nfdc route add /client ether://[0A:5C:7E:54:66:E7]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[2].execute('ndnsec key-gen /hydra/node3 | ndnsec cert-install -')
    print(stdout) 
except Exception as e:
    print(f"Exception: {e}")


Node 2
face-updated id=270 local=dev://ens7 remote=ether://[26:cb:8b:88:5b:c0] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node2 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/node2 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

Node 4
face-updated id=269 local=dev://ens9 remote=ether://[4e:ca:e2:44:72:54] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=269 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node4 nexthop=269 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted

In [33]:
# face and route node4
try:
    stdout, stderr = slice.get_nodes()[3].execute('nfd-start')
    print(stdout)
    
    print("Node 2")
    stdout, stderr = slice.get_nodes()[3].execute('nfdc face create remote ether://[2A:F7:3B:95:6C:4F] local dev://ens8')              
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /hydra/group ether://[2A:F7:3B:95:6C:4F]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /hydra/node/node2 ether://[2A:F7:3B:95:6C:4F]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /node2 ether://[2A:F7:3B:95:6C:4F]')              
    print(stdout)
    
    print("Node 3")
    stdout, stderr = slice.get_nodes()[3].execute('nfdc face create remote ether://[3A:4A:C8:FF:0B:1C] local dev://ens9')              
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /hydra/group ether://[3A:4A:C8:FF:0B:1C]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /hydra/node/node3 ether://[3A:4A:C8:FF:0B:1C]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /node3 ether://[3A:4A:C8:FF:0B:1C]')              
    print(stdout)
    
    print("Node 5")
    stdout, stderr = slice.get_nodes()[3].execute('nfdc face create ether://[5A:69:88:1B:A0:8D] local dev://ens10 ')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /hydra/group ether://[5A:69:88:1B:A0:8D]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /hydra/node/node5 ether://[5A:69:88:1B:A0:8D]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /node5 ether://[5A:69:88:1B:A0:8D]')
    
    print("Node 1")
    stdout, stderr = slice.get_nodes()[3].execute('nfdc strategy set /hydra/group /localhost/nfd/strategy/multicast')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[3].execute('nfdc face create ether://[0A:5C:7E:54:66:E7] local dev://ens7')
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('nfdc route add /client ether://[0A:5C:7E:54:66:E7]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[3].execute('ndnsec key-gen /hydra/node4 | ndnsec cert-install -')
    print(stdout) 
except Exception as e:
    print(f"Exception: {e}")


Node 2
face-created id=270 local=dev://ens8 remote=ether://[2a:f7:3b:95:6c:4f] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node2 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/node2 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

Node 3
face-created id=275 local=dev://ens9 remote=ether://[3a:4a:c8:ff:0b:1c] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=275 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node3 nexthop=275 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted

In [34]:
# face and route node5
try:
    stdout, stderr = slice.get_nodes()[4].execute('nfd-start')
    print(stdout)
    
    print("Node 2")
    stdout, stderr = slice.get_nodes()[4].execute('nfdc face create remote ether://[26:9B:CA:79:F5:4C] local dev://ens9')             
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /hydra/group ether://[26:9B:CA:79:F5:4C]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /hydra/node/node2 ether://[26:9B:CA:79:F5:4C]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /node2 ether://[26:9B:CA:79:F5:4C]')              
    print(stdout)
    
    print("Node 3")
    stdout, stderr = slice.get_nodes()[4].execute('nfdc face create remote ether://[42:D0:91:43:72:DC] local dev://ens10')              
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /hydra/group ether://[42:D0:91:43:72:DC]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /hydra/node/node3 ether://[42:D0:91:43:72:DC]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /node3 ether://[42:D0:91:43:72:DC]')              
    print(stdout)
    
    print("Node 4")
    stdout, stderr = slice.get_nodes()[4].execute('nfdc face create remote ether://[52:B9:E3:B4:2E:A5] local dev://ens8')
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /hydra/group ether://[52:B9:E3:B4:2E:A5]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /hydra/node/node4 ether://[52:B9:E3:B4:2E:A5]')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /node4 ether://[52:B9:E3:B4:2E:A5]')
    print(stdout)
    
    print("Node 1")
    stdout, stderr = slice.get_nodes()[4].execute('nfdc strategy set /hydra/group /localhost/nfd/strategy/multicast')
    print(stdout) 
    stdout, stderr = slice.get_nodes()[4].execute('nfdc face create ether://[0A:5C:7E:54:66:E7] local dev://ens7')
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('nfdc route add /client ether://[0A:5C:7E:54:66:E7]')
    print(stdout)
    stdout, stderr = slice.get_nodes()[4].execute('ndnsec key-gen /hydra/node5 | ndnsec cert-install -')
    print(stdout) 
except Exception as e:
    print(f"Exception: {e}")


Node 2
face-created id=270 local=dev://ens9 remote=ether://[26:9b:ca:79:f5:4c] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node2 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/node2 nexthop=270 origin=static cost=0 flags=child-inherit expires=never

Node 3
face-created id=275 local=dev://ens10 remote=ether://[42:d0:91:43:72:dc] persistency=persistent reliability=off congestion-marking=off congestion-marking-interval=100ms default-congestion-threshold=65536B mtu=1500

route-add-accepted prefix=/hydra/group nexthop=275 origin=static cost=0 flags=child-inherit expires=never

route-add-accepted prefix=/hydra/node/node3 nexthop=275 origin=static cost=0 flags=child-inherit expires=never

route-add-accepte

### update and renew the slice 

In [5]:
import datetime

#Set end host to now plus 1 day
end_date = (datetime.datetime.now() + datetime.timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

## Delete Slice

In [20]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")